In [1]:
from IPython.display import display, HTML

display(
    HTML("""
<style>
* {font-family:D2Coding;}
div.container{width:87% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input { font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:3px;}
table.dataframe{font-size:12px;}
</style>
"""))

<font color="red" size="5">ch15. 데이터베이스 연동</font>
# 1절. SQLite 데이터 베이스 연동
- SQLite 데이터베이스는 별도의 DBMS없이 SQL을 이용하여 DB 엑세스 가능한 디스크 기반 DB
- SQLite는 프로토타입(시제품) 용도
- 프로젝트 단계 : 분석 - 설계 - 구현 - 테스트 - 고객에게 배포 - 유지보수
-                 SQLite    Oracle/MySql, MariaDB, ...
- C 라이브러리
- [db browser for sqlite](https://sqlitebrowser.org/)
## 1.1 SQLite 패키지 load

In [3]:
import sqlite3
sqlite3.sqlite_version

'3.40.1'

In [4]:
import pandas as pd
pd.__version__

'1.5.3'

## 1.2 데이터 베이스 연결

In [5]:
# DB 연결
conn = sqlite3.connect('data/ch15_exmple.db')
conn

In [6]:
# 커서객체(SQL문 실행->결과 조회) 생성
cursor = conn.cursor()
cursor

In [8]:
cursor.execute('''
    CREATE TABLE MEMBER(
        NAME TEXT,
        AGE  INT,
        EMAIL TEXT
    )
''')

### insert, update, delete 전송

In [7]:
cursor.execute('''
    INSERT INTO MEMBER VALUES ('홍길동', 20, 'h@h.com')
''')
print('수행 결과 행수 :', cursor.rowcount)

수행 결과 행수 : 1


In [8]:
sql = "INSERT INTO MEMBER VALUES ('박길동', 30, 'k@h.com')"
print(sql)
cursor.execute(sql)
print('수행 결과 행수 :', cursor.rowcount)

INSERT INTO MEMBER VALUES ('박길동', 30, 'k@h.com')
수행 결과 행수 : 1


In [9]:
conn.commit() # (반) conn.rollback() DML에서만 가능

### select 전송

In [14]:
cursor.execute('''
    SELECT NAME, AGE+1 NEXTAGE, EMAIL FROM MEMBER
''')

In [15]:
# SELECT 전송
#   select 문 실행결과를 받는 함수
#    cursor.fetchone() : 결과를 한행씩 받을 때(튜플)
#    cursor.fetchall() : 결과를 모두 받을 때 (튜플 list)
#    cursor.fetchmany(n) : 결과를 n행 받을 때(튜플 list)
#    cursor.description : header 내용을 포함한 내용들(list)

# INSERT, UPDATE, DELETE 전송 : cursor.rowcount
print(cursor.fetchall())

[('홍길동', 21, 'h@h.com'), ('김길동', 31, 'k@h.com'), ('마길동', 31, 'k@h.com'), ('윤길동', 31, 'k@h.com'), ('박길동', 31, 'k@h.com'), ('이순신', 55, 'lee@korea.com'), ('홍길동', 21, 'h@h.com'), ('박길동', 31, 'k@h.com')]


In [16]:
print(cursor.fetchall())

[]


In [17]:
cursor.execute('''
    SELECT NAME, AGE+1 NEXTAGE, EMAIL FROM MEMBER order by NEXTAGE
''')
members = cursor.fetchmany(3)
members

[('홍길동', 21, 'h@h.com'), ('홍길동', 21, 'h@h.com'), ('김길동', 31, 'k@h.com')]

In [18]:
cursor.execute('''
    SELECT NAME, AGE+1 NEXTAGE, EMAIL FROM MEMBER order by NEXTAGE
''')
members = cursor.fetchone()
members

('홍길동', 21, 'h@h.com')

In [24]:
cursor.fetchall()

[('김길동', 31, 'k@h.com'),
 ('마길동', 31, 'k@h.com'),
 ('윤길동', 31, 'k@h.com'),
 ('박길동', 31, 'k@h.com')]

In [19]:
# 데이터 한줄씩 가져오기
cursor.execute('''
    SELECT NAME, AGE+1 NEXTAGE, EMAIL FROM MEMBER order by NEXTAGE
''')
members = []
while True:
    member = cursor.fetchone()
    if member is None:
        break
    print(member)
    members.append({'name':member[0], 'age':member[1], 'email':member[2]})
import pandas as pd
pd.DataFrame(members)

('홍길동', 21, 'h@h.com')
('홍길동', 21, 'h@h.com')
('김길동', 31, 'k@h.com')
('마길동', 31, 'k@h.com')
('윤길동', 31, 'k@h.com')
('박길동', 31, 'k@h.com')
('박길동', 31, 'k@h.com')
('이순신', 55, 'lee@korea.com')


,name,age,email
0,홍길동,21,h@h.com
1,홍길동,21,h@h.com
2,김길동,31,k@h.com
3,마길동,31,k@h.com
4,윤길동,31,k@h.com
5,박길동,31,k@h.com
6,박길동,31,k@h.com
7,이순신,55,lee@korea.com


In [20]:
cursor.execute('''
    SELECT NAME, AGE+1 NEXTAGE, EMAIL FROM MEMBER order by NEXTAGE
''')
members = cursor.fetchall()
df = pd.DataFrame(members, 
                  columns=[description[0] for description in cursor.description])
df

,NAME,NEXTAGE,EMAIL
0,홍길동,21,h@h.com
1,홍길동,21,h@h.com
2,김길동,31,k@h.com
3,마길동,31,k@h.com
4,윤길동,31,k@h.com
5,박길동,31,k@h.com
6,박길동,31,k@h.com
7,이순신,55,lee@korea.com


In [21]:
# select문을 수행한 필드 정보
cursor.description

(('NAME', None, None, None, None, None, None),
 ('NEXTAGE', None, None, None, None, None, None),
 ('EMAIL', None, None, None, None, None, None))

In [22]:
[description[0] for description in cursor.description]

['NAME', 'NEXTAGE', 'EMAIL']

## 1.3 SQL구문에 파라미터 사용하기
- qmark(DB에 따라 불가한 경우 있음)
- named(추천)

In [23]:
conn = sqlite3.connect('data/ch15_exmple.db')
cursor = conn.cursor()
cursor.execute("SELECT * FROM MEMBER WHERE NAME IN ('홍길동','김길동')")
cursor.fetchall()

[('홍길동', 20, 'h@h.com'), ('김길동', 30, 'k@h.com'), ('홍길동', 20, 'h@h.com')]

In [24]:
# 파라미터 사용하기 : qmark 방법 이용
sql = "SELECT * FROM MEMBER WHERE NAME IN (?,?)"
name1 = input('검색할 이름은?')
name2 = input('검색할 다른이름은?')
cursor.execute(sql, (name1, name2))
cursor.fetchall()

검색할 이름은?홍길동
검색할 다른이름은?박길동


[('홍길동', 20, 'h@h.com'),
 ('박길동', 30, 'k@h.com'),
 ('홍길동', 20, 'h@h.com'),
 ('박길동', 30, 'k@h.com')]

In [35]:
# 파라미터 사용하기 : named 방법 이용
sql = "SELECT * FROM MEMBER WHERE NAME IN (:name1, :name2)"
name1 = input('검색할 이름은?')
name2 = input('검색할 다른이름은?')
cursor.execute(sql, {'name1':name1, 'name2':name2})
cursor.fetchall()

검색할 이름은?홍길동
검색할 다른이름은?김길동


[('홍길동', 20, 'h@h.com'), ('김길동', 30, 'k@h.com')]

In [37]:
# named 방법으로 데이터 입력하기
try:
    name = input('입력할 이름은?')
    age = int(input('입력할 나이는?'))
except ValueError:
    print('유효하지 않은 나이를 입력하시면 18세로 초기화')
    age = 18
finally:
    email = input('입력할 메일은?')
cursor.execute('INSERT INTO MEMBER VALUES (:name, :age, :email)', 
               {'name':name, 'age':age, 'email':email})
conn.commit()
if cursor.rowcount == 1:
    print('입력성공')
else:
    print('입력실패')

입력할 이름은?이순신
입력할 나이는?54
입력할 메일은?lee@korea.com
입력성공


In [38]:
cursor.close()
conn.close()

# 2절. 오라클 데이터 베이스 연결
- pip install cx_oracle

In [25]:
import cx_Oracle
# conn 얻어오는 방법1
conn = cx_Oracle.connect('scott','tiger','localhost:1521/xe')
conn

<cx_Oracle.Connection to scott@localhost:1521/xe>

In [26]:
# conn 얻어오는 방법2
oracle_dsn = cx_Oracle.makedsn(host='localhost',port=1521, sid="xe")
conn = cx_Oracle.connect("scott", "tiger", dsn=oracle_dsn)
conn

<cx_Oracle.Connection to scott@(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=localhost)(PORT=1521))(CONNECT_DATA=(SID=xe)))>

In [27]:
cursor = conn.cursor()
cursor.execute('SELECT EMPNO NO, ENAME, JOB, MGR, SAL, COMM FROM EMP')
emps = cursor.fetchall()
for emp in emps:
    print(emp)

(7369, 'SMITH', 'CLERK', 7902, 800.0, None)
(7499, 'ALLEN', 'SALESMAN', 7698, 1600.0, 300.0)
(7521, 'WARD', 'SALESMAN', 7698, 1250.0, 500.0)
(7566, 'JONES', 'MANAGER', 7839, 2975.0, None)
(7654, 'MARTIN', 'SALESMAN', 7698, 1250.0, 1400.0)
(7698, 'BLAKE', 'MANAGER', 7839, 2850.0, None)
(7782, 'CLARK', 'MANAGER', 7839, 2450.0, None)
(7788, 'SCOTT', 'ANALYST', 7566, 3000.0, None)
(7839, 'KING', 'PRESIDENT', None, 5000.0, None)
(7844, 'TURNER', 'SALESMAN', 7698, 1500.0, 0.0)
(7876, 'ADAMS', 'CLERK', 7788, 1100.0, None)
(7900, 'JAMES', 'CLERK', 7698, 950.0, None)
(7902, 'FORD', 'ANALYST', 7566, 3000.0, None)
(7934, 'MILLER', 'CLERK', 7782, 1300.0, None)


In [28]:
pd.DataFrame(emps,
            columns=[des[0] for des in cursor.description])

,NO,ENAME,JOB,MGR,SAL,COMM
0,7369,SMITH,CLERK,7902.0,800.0,NaN
1,7499,ALLEN,SALESMAN,7698.0,1600.0,300.0
2,7521,WARD,SALESMAN,7698.0,1250.0,500.0
3,7566,JONES,MANAGER,7839.0,2975.0,NaN
4,7654,MARTIN,SALESMAN,7698.0,1250.0,1400.0
5,7698,BLAKE,MANAGER,7839.0,2850.0,NaN
6,7782,CLARK,MANAGER,7839.0,2450.0,NaN
7,7788,SCOTT,ANALYST,7566.0,3000.0,NaN
8,7839,KING,PRESIDENT,NaN,5000.0,NaN
9,7844,TURNER,SALESMAN,7698.0,1500.0,0.0
